In [110]:
import numpy as np
import numpy.linalg as la 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import pymc3 as pm

# plt.rcParams['figure.figsize'] = [16,8]
# plt.rcParams.update({'font.size': 18})
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [111]:
dataf = pd.read_csv('premier_league_2013_2014.dat', sep=',', header=None)
dataf.columns = ['goals_home', 'goals_away', 'home_team', 'away_team']

In [123]:
def enforce_corner_constraint(theta, n_teams):
    theta[0] = theta[n_teams] = 0
    return theta

def propose_samples(current, sigma):
    I = np.eye(current.shape[0])
    proposal = np.random.multivariate_normal(current, sigma*I)
    return proposal

def compute_hyper_prior_probabilities(eta):
    tau1 = 1/0.0001
    alpha = beta = 0.1

    mu_attack_logprob = st.norm(0, 1/np.sqrt(tau1)).logpdf(eta[0])
    mu_defense_logprob = st.norm(0, 1/np.sqrt(tau1)).logpdf(eta[1])
    tau_attack_logprob = st.gamma(alpha, scale=1/beta).logpdf(eta[2])
    tau_defense_logprob = st.gamma(alpha, scale=1/beta).logpdf(eta[3])

    eta_logprob = mu_attack_logprob + mu_defense_logprob + tau_attack_logprob + tau_defense_logprob
    return eta_logprob

def compute_prior_probabilities(eta, theta, n_teams):
    tau0 = 1/0.0001

    mu_att = eta[0]
    mu_def = eta[1]
    tau_att = eta[2]
    tau_def = eta[3]
    
    home = theta[-1]
    attack = theta[:n_teams]
    defense = theta[n_teams:-1]

    home_logprob = st.norm(0, 1/np.sqrt(tau0)).logpdf(home)
    attack_logprob = st.norm(mu_att, 1/np.sqrt(tau_att)).logpdf(attack)
    defense_logprob = st.norm(mu_def, 1/np.sqrt(tau_def)).logpdf(defense)

    theta_prob = home_logprob + np.sum(attack_logprob + defense_logprob)
    return theta_prob

def compute_likelihood(theta, n_teams, dataf):

    goals_home = dataf['goals_home'].values.astype(int)
    goals_away = dataf['goals_away'].values.astype(int)
    home_team = dataf['home_team'].values.astype(int)
    away_team = dataf['away_team'].values.astype(int)

    home = theta[-1]
    attack = theta[:n_teams]
    defense = theta[n_teams:-1]

    theta_home = np.exp(home + attack[home_team] - defense[away_team])
    theta_away = np.exp(attack[away_team] - defense[home_team])
    
    loglikelihood_home = st.poisson(theta_home).logpmf(goals_home)
    loglikelihood_away = st.poisson(theta_away).logpmf(goals_away)

    loglikelihood = np.sum(loglikelihood_home + loglikelihood_away)
    return loglikelihood

def compute_probabilities(eta, theta, n_teams, dataf):
    eta_logprob = compute_hyper_prior_probabilities(eta)
    theta_logprob = compute_prior_probabilities(eta, theta, n_teams)
    loglikelihood = compute_likelihood(theta, n_teams, dataf)
    logprob = eta_logprob + theta_logprob + loglikelihood
    return logprob

def metropolis_hastings(n_samples, sigma, dataf, thinning, burn_in):

    n_teams = 20
    starting_point = 0.1
    eta_current = np.full(4, starting_point)
    theta_current = enforce_corner_constraint(np.full(1+n_teams*2, starting_point), n_teams)
    samples = []
    
    for t in range(n_samples):

        eta_proposal = propose_samples(eta_current, sigma)
        theta_proposal = enforce_corner_constraint(propose_samples(theta_current, sigma), n_teams)

        current_logprob = compute_probabilities(eta_current, theta_current, n_teams, dataf)
        proposal_logprob = compute_probabilities(eta_proposal, theta_proposal, n_teams, dataf)
        acceptance_logprob = proposal_logprob - current_logprob

        u = np.random.uniform()
        if np.log(u) < acceptance_logprob:
            eta_current = eta_proposal.copy()
            theta_current = theta_proposal.copy()
        
        if (t % thinning == 0) and (t > burn_in):
            samples.append([eta_current.copy(), theta_current.copy()])

    return samples

In [124]:
n_samples = 10
sigmas = [0.005, 0.05, 0.5]
thinning = [1, 5, 20, 50]
burn_in = 10

for i in range(len(sigmas)):
    samples = metropolis_hastings(n_samples, sigmas[i], dataf, thinning[i], burn_in)
    break

C:\Users\erikn\AppData\Local\Temp\ipykernel_388\1367081123.py:35: RuntimeWarning: invalid value encountered in sqrt
  attack_logprob = st.norm(mu_att, 1/np.sqrt(tau_att)).logpdf(attack)


In [106]:
with pm.Model() as model:

    n_teams = 20
    n_games = 380

    tau0 = 1/0.0001
    tau1 = 1/0.0001
    alpha = beta = 1

    mu_attack = np.random.normal(0, 1/np.sqrt(tau1)) 
    mu_defense = np.random.normal(0, 1/np.sqrt(tau1)) 
    tau_attack = np.random.gamma(alpha, scale=1/beta)
    tau_defense = np.random.gamma(alpha, scale=1/beta)

    home = np.random.normal(0, 1/np.sqrt(tau0))
    attack = np.random.normal(mu_attack, 1/np.sqrt(tau_attack), n_teams)
    defense = np.random.normal(mu_defense, 1/np.sqrt(tau_defense), n_teams)

    attack[0] = 0
    defense[0] = 0

    theta_home = np.exp(home + attack[ht] + defense[at])
    theta_away = np.exp(attack[at] + defense[ht])

    print(home, attack, defense)

    goals_home = pm.Poisson('goals_home', mu=theta_home, observed=df['goals_home'])
    goals_away = pm.Poisson('goals_away', mu=theta_away, observed=df['goals_away'])

    trace = pm.sample(5000, tune=1000)

# pm.summary(trace).round(2)

0.0005999435130517092 [ 0.          0.9193612   0.75510683  0.12444181  0.25732103  0.02161338
  0.7972385  -0.10846091  0.24090788  0.41470819 -0.29768518 -0.1282222
  0.33784883 -0.51661573 -0.51086637  0.98380653 -1.00928912  0.16073458
  0.30995141 -1.99319773] [ 0.         -0.91728703  0.50857573  0.33846646  0.48485596 -0.63676431
 -4.28147731  2.39820147  1.53051061 -0.70307521  2.07114262 -2.60504444
 -0.38097132  2.42968423  0.12095222 -1.42639252  2.29115609 -1.58943519
  0.83696372  0.97117246]


ValueError: The model does not contain any free variables.